# Training on multiple GPUs

``` text
The tf.distribute.Strategy API provides an abstraction for distributing your training across multiple processing units. It allows you to carry out distributed training using existing models and training code with minimal changes.

This tutorial demonstrates how to use the tf.distribute.MirroredStrategy to perform in-graph replication with synchronous training on many GPUs on one machine. The strategy essentially copies all of the model's variables to each processor. Then, it uses all-reduce to combine the gradients from all processors, and applies the combined value to all copies of the model.

You will use the tf.keras APIs to build the model and Model.fit for training it. (To learn about distributed training with a custom training loop and the MirroredStrategy, check out this tutorial.)

MirroredStrategy trains your model on multiple GPUs on a single machine. For synchronous training on many GPUs on multiple workers, use the tf.distribute.MultiWorkerMirroredStrategy with the Keras Model.fit or a custom training loop. For other options, refer to the Distributed training guide.

To learn about various other strategies, there is the Distributed training with TensorFlow guide.

```

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

import os

# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [2]:
print(tf.__version__)

2.7.0


In [3]:
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = datasets['train'], datasets['test']

2021-09-15 11:03:56.839905: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /home/maker/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


2021-09-15 11:04:03.861211: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 47220 MB memory:  -> device: 0, name: Quadro RTX 8000, pci bus id: 0000:67:00.0, compute capability: 7.5
2021-09-15 11:04:03.863975: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46893 MB memory:  -> device: 1, name: Quadro RTX 8000, pci bus id: 0000:68:00.0, compute capability: 7.5


## Define the distribution strategy

``` text 
Create a MirroredStrategy object. This will handle distribution and provide a context manager (MirroredStrategy.scope) to build your model inside.

```

In [4]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [5]:
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 2


## Set up the input pipeline

When training a model with multiple GPUs, you can use the extra computing power effectively by increasing the batch size. In general, use the largest batch size that fits the GPU memory and tune the learning rate accordingly.

In [7]:
# You can also do info.splits.total_num_examples to get the total
# number of examples in the dataset.

num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

In [8]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255

  return image, label

In [9]:
train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

In [11]:
print (num_train_examples, num_test_examples)

60000 10000


## Create the model

In [12]:
with strategy.scope():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10)
  ])

  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


## Define the callbacks

Define the following tf.keras.callbacks:

* tf.keras.callbacks.TensorBoard: writes a log for TensorBoard, which allows you to visualize the graphs.
* tf.keras.callbacks.ModelCheckpoint: saves the model at a certain frequency, such as after every epoch.
* tf.keras.callbacks.LearningRateScheduler: schedules the learning rate to change after, for example, every epoch/batch.

For illustrative purposes, add a custom callback called PrintLR to display the learning rate in the notebook.

In [13]:
# Define the checkpoint directory to store the checkpoints.
checkpoint_dir = './training_checkpoints'
# Define the name of the checkpoint files.
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

# Define a function for decaying the learning rate.
# You can define any decay function you need.
def decay(epoch):
  if epoch < 3:
    return 1e-3
  elif epoch >= 3 and epoch < 7:
    return 1e-4
  else:
    return 1e-5

# Define a callback for printing the learning rate at the end of each epoch.
class PrintLR(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    print('\nLearning rate for epoch {} is {}'.format(epoch + 1,
                                                      model.optimizer.lr.numpy()))
    
# Put all the callbacks together.
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                       save_weights_only=True),
    tf.keras.callbacks.LearningRateScheduler(decay),
    PrintLR()
]

2021-09-15 11:10:10.391642: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2021-09-15 11:10:10.391706: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2021-09-15 11:10:10.394080: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1630] Profiler found 2 GPUs
2021-09-15 11:10:10.394882: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.4'; dlerror: libcupti.so.11.4: cannot open shared object file: No such file or directory
2021-09-15 11:10:10.395058: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so'; dlerror: libcupti.so: cannot open shared object file: No such file or directory
2021-09-15 11:10:10.395083: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1681] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be 

In [14]:
## Train and evaluate

EPOCHS = 12

model.fit(train_dataset, epochs=EPOCHS, callbacks=callbacks)


2021-09-15 11:10:27.276717: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:466] The `assert_cardinality` transformation is currently not handled by the auto-shard rewrite and will be removed.


Epoch 1/12
INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


...

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
2021-09-15 11:10:32.780614: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202
2021-09-15 11:10:33.425817: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202


  3/469 [..............................] - ETA: 33s - loss: 2.2444 - accuracy: 0.1901   WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0062s vs `on_train_batch_end` time: 0.0220s). Check your callbacks.


2021-09-15 11:10:37.958733: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2021-09-15 11:10:37.958759: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2021-09-15 11:10:37.958795: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1681] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
2021-09-15 11:10:37.971328: I tensorflow/core/profiler/lib/profiler_session.cc:67] Profiler session collecting data.
2021-09-15 11:10:37.971652: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1773] function cupti_interface_->Finalize()failed with error CUPTI could not be loaded or symbol could not be found.
2021-09-15 11:10:37.988446: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:526]  GpuTracer has collected 0 callback api events and 0 activity events. 
2021-09-15 11:10:37.994946: I tensorflow

469/469 [==============================] - 14s 8ms/step - loss: 0.2622 - accuracy: 0.9266

Learning rate for epoch 1 is 0.0010000000474974513
Epoch 2/12
469/469 [==============================] - 3s 7ms/step - loss: 0.0894 - accuracy: 0.9742

Learning rate for epoch 2 is 0.0010000000474974513
Epoch 3/12
469/469 [==============================] - 3s 7ms/step - loss: 0.0622 - accuracy: 0.9819

Learning rate for epoch 3 is 0.0010000000474974513
Epoch 4/12
469/469 [==============================] - 3s 7ms/step - loss: 0.0400 - accuracy: 0.9891

Learning rate for epoch 4 is 9.999999747378752e-05
Epoch 5/12
469/469 [==============================] - 3s 7ms/step - loss: 0.0372 - accuracy: 0.9896

Learning rate for epoch 5 is 9.999999747378752e-05
Epoch 6/12
469/469 [==============================] - 3s 7ms/step - loss: 0.0353 - accuracy: 0.9906

Learning rate for epoch 6 is 9.999999747378752e-05
Epoch 7/12
469/469 [==============================] - 3s 7ms/step - loss: 0.0336 - accuracy: 0.991

In [15]:
# Check the checkpoint directory... run this on your shell 
ls {checkpoint_dir}

SyntaxError: invalid syntax (3647772983.py, line 2)

In [16]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

eval_loss, eval_acc = model.evaluate(eval_dataset)

print('Eval loss: {}, Eval accuracy: {}'.format(eval_loss, eval_acc))

2021-09-15 11:12:11.166267: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:466] The `assert_cardinality` transformation is currently not handled by the auto-shard rewrite and will be removed.


79/79 [==============================] - 2s 6ms/step - loss: 0.0441 - accuracy: 0.9852
Eval loss: 0.044124308973550797, Eval accuracy: 0.9851999878883362


In [17]:
%tensorboard --logdir=logs

## Export to SavedModel

In [18]:
path = 'saved_model/'
model.save(path, save_format='tf')

2021-09-15 11:14:38.044196: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


now, load the model without Stategy.scope:

In [19]:
unreplicated_model = tf.keras.models.load_model(path)

unreplicated_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy'])

eval_loss, eval_acc = unreplicated_model.evaluate(eval_dataset)

print('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

79/79 [==============================] - 0s 3ms/step - loss: 0.0441 - accuracy: 0.9852
Eval loss: 0.044124308973550797, Eval Accuracy: 0.9851999878883362


Load the model with Strategy.scope:

In [20]:
with strategy.scope():
  replicated_model = tf.keras.models.load_model(path)
  replicated_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                           optimizer=tf.keras.optimizers.Adam(),
                           metrics=['accuracy'])

  eval_loss, eval_acc = replicated_model.evaluate(eval_dataset)
  print ('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

2021-09-15 11:16:28.883440: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:466] The `assert_cardinality` transformation is currently not handled by the auto-shard rewrite and will be removed.


79/79 [==============================] - 3s 5ms/step - loss: 0.0441 - accuracy: 0.9852
Eval loss: 0.044124308973550797, Eval Accuracy: 0.9851999878883362
